### Import libraries

In [1]:
! pip install netCDF4 # python API to work with netcdf (.nc) files

Defaulting to user installation because normal site-packages is not writeable


In [1]:
import netCDF4 # python API to work with netcdf (.nc) files
import os # function as isFile?
import math  # mathematic operations - sqrt,..
import datetime #for for with date varibles
import numpy as np # library to work with matrixes and computations in general
import ray  #library for parallelization
ray.init()
from auxiliary_classes import convert_time,convert_time_reverse,kelvin_to_celsius,kelvin_to_celsius_vector,Grid,Image,subImage  # auxiliary function for working with netCDF4


2021-01-03 15:28:01,050	INFO services.py:1173 -- View the Ray dashboard at http://127.0.0.1:8265


### CDS API

In [2]:
import cdsapi # CDS API
!echo "url: https://cds.climate.copernicus.eu/api/v2" > ~/.cdsapirc
!echo "key: 10359:3de9079e-99e8-4c32-885c-72967196c52b" >> ~/.cdsapirc
c = cdsapi.Client()

### Auxiliary functions:

Download ERA5-Land and EDA for specific location and one year:

In [3]:
#Request EDA:
@ray.remote #parallelization 
def retrieveeda(edadataset,vartemp, varevapotranspiration, varrunoff, varsoiltemp, varsolar, varprecipitation,syear,max_lat,min_lon,min_lat,max_lon,edaname):
    c.retrieve( 
        edadataset,
          {
                'product_type': ['ensemble_mean', 'ensemble_spread'],
                'variable': [vartemp, varevapotranspiration, varrunoff, varsoiltemp, varsolar, varprecipitation],
                'year': [syear],
                'month': [
                    '01', '02', '03',
                    '04', '05', '06',
                    '07', '08', '09',
                    '10', '11', '12',
                ],
                'day': [
                    '01', '02', '03',
                    '04', '05', '06',
                    '07', '08', '09',
                    '10', '11', '12',
                    '13', '14', '15',
                    '16', '17', '18',
                    '19', '20', '21',
                    '22', '23', '24',
                    '25', '26', '27',
                    '28', '29', '30',
                    '31'],
                'time': [
                    '00:00', '01:00', '02:00',
                    '03:00', '04:00', '05:00',
                    '06:00', '07:00', '08:00',
                    '09:00', '10:00', '11:00',
                    '12:00', '13:00', '14:00',
                    '15:00', '16:00', '17:00',
                    '18:00', '19:00', '20:00',
                    '21:00', '22:00', '23:00'],
                'area': [max_lat,min_lon,min_lat,max_lon],
                'format':'netcdf'
            },
        edaname)        

In [4]:
#Request ERA:
@ray.remote #parallelization 
def retrieveera(eradataset,vartemp, varevapotranspiration, varrunoff, varsoiltemp, varsolar, varprecipitation,syear,max_lat,min_lon,min_lat,max_lon,eraname):
    c.retrieve(
        eradataset,
            {
            'variable': [vartemp, varevapotranspiration, varrunoff,  varsoiltemp,  varsolar, varprecipitation],
            'year':[syear],
            'month':[
                '01','02','03',
                '04','05','06',
                '07','08','09',
                '10','11','12'],
            'day':[
                '01','02','03',
                '04','05','06',
                '07','08','09',
                '10','11','12',
                '13','14','15',
                '16','17','18',
                '19','20','21',
                '22','23','24',
                '25','26','27',
                '28','29','30',
                '31'],
            'time':[
                '00:00','01:00','02:00',
                '03:00','04:00','05:00',
                '06:00','07:00','08:00',
                '09:00','10:00','11:00',
                '12:00','13:00','14:00',
                '15:00','16:00','17:00',
                '18:00','19:00','20:00',
                '21:00','22:00','23:00'],
            'area': [max_lat,min_lon,min_lat,max_lon],
            'format':'netcdf'
            },
        eraname)

In [5]:
# Download ERA and EDA of one year:
def oneyearapi(year,max_lat,min_lat, max_lon, min_lon,era5fname, edafname,eradataset,edadataset,vartemp,varevapotranspiration, varrunoff, varsoiltemp, varsolar, varprecipitation,folder):
    syear = str(year)
    # EDA:
    edadataset =  edadataset
    edaname = folder + "/" + edafname    
    # ERA:
    eradataset =  eradataset
    eraname = folder + "/" + era5fname    
    # ParalLelization:
    ray.get([retrieveeda.remote(edadataset,vartemp, varevapotranspiration, varrunoff, varsoiltemp, varsolar, varprecipitation,syear,max_lat,min_lon,min_lat,max_lon,edaname), retrieveera.remote(eradataset,vartemp, varevapotranspiration, varrunoff, varsoiltemp, varsolar, varprecipitation,syear,max_lat,min_lon,min_lat,max_lon,eraname)])
       

Export to csv:

In [6]:
#For printing to csv: 
def print_csv(foldername,fname, ttext, startdoc): 
    fname = fname +".csv"
    if foldername != 0: # 0 = no output folder
        fname = foldername + "/" + fname
    pmode="a" # a = mode for writing, w = mode for creating new file
    if startdoc==1:
        with open(fname, mode="w", encoding='utf-8') as f1: #start of csv
            tstring = ttext 
            print(tstring, file=f1)
            f1.close()
    else:
        with open(fname, mode=pmode, encoding='utf-8') as f1: #write line to csv
                tstring = ttext 
                print(tstring, file=f1)
                f1.close()

Auxiliary functions for calculations:

In [7]:
#Calculate horizontal distance:
def distancecalculation(x1,y1,x2,y2): 
    result = math.sqrt(((x1-x2)*(x1-x2))+((y1-y2)*(y1-y2)))
    return result      

In [8]:
#Find 4 nearest EDA points - latitudes, longitudes:
def findnearpoints(latitude,longitude,lat2list,lon2list): 
        latmin=0
        latmax=0
        lonmin=0
        lonmax=0
        listlatlen = len(lat2list)
        listlonlen = len(lon2list)
        for i in range(0, listlatlen, 1):
            currentlat = lat2list[i] 
            if latitude == currentlat:
                latmin = currentlat
                latmax = currentlat
                break
            elif latitude < currentlat:
                latmin = lat2list[i-1]
                latmax = currentlat
                break
        for i in range(0, listlonlen, 1):
            currentlon = lon2list[i] 
            if longitude == currentlon:
                lonmin = currentlon
                lonmax = currentlon
                break
            elif longitude < currentlon:
                lonmin = lon2list[i-1]
                lonmax = currentlon
                break            
        return latmin,latmax,lonmin,lonmax

In [9]:
#Find EDA value for place from 4 nearest points, using IDW (power parameter = 2):
def edahourvalue(latitude, longitude,lat2list,lon2list,ie,edaparameter,longitudeparam, latitudeparam, timeparam, year, month, day, hour): 
    finaleda = 0
    listlonlat = []
    listlonlat = findnearpoints(latitude, longitude,lat2list,lon2list)
    if listlonlat[0]==listlonlat[1] and listlonlat[2]==listlonlat[3]:
        time=convert_time_reverse(datetime.datetime(year, month, day, hour, 0))
        slice_dictionary={longitudeparam:[longitude],latitudeparam:[latitude],timeparam:[int(time)]}
        currenttemp=ie.slice(edaparameter,slice_dictionary)
        finaleda=float(currenttemp)
    elif listlonlat[0]==listlonlat[1] and listlonlat[2]!=listlonlat[3]:
        lat1 = listlonlat[0] 
        lat2 = listlonlat[1]
        lon1 = listlonlat[2]
        lon2 = listlonlat[3]
        dist1 = distancecalculation(latitude,longitude,lat1,lon1)
        dist2 = distancecalculation(latitude,longitude,lat2,lon2)
        time=convert_time_reverse(datetime.datetime(year, month, day, hour, 0))
        slice_dictionary={longitudeparam:[lon1],latitudeparam:[lat1],timeparam:[int(time)]}
        currenttemp=ie.slice(edaparameter,slice_dictionary)
        val1=float(currenttemp)
        slice_dictionary={longitudeparam:[lon2],latitudeparam:[lat2],timeparam:[int(time)]}
        currenttemp=ie.slice(edaparameter,slice_dictionary)
        val2=float(currenttemp)
           
        idw1 = (val1/(dist1*dist1))+(val2/(dist2*dist2))
        idw2 = (1/(dist1*dist1))+(1/(dist2*dist2))
        idw = idw1/idw2
        finaleda = idw
    elif listlonlat[0]!=listlonlat[1] and listlonlat[2]==listlonlat[3]:
        lat1 = listlonlat[0] 
        lat2 = listlonlat[1]
        lon1 = listlonlat[2]
        lon2 = listlonlat[3]
        dist1 = distancecalculation(latitude,longitude,lat1,lon1)
        dist2 = distancecalculation(latitude,longitude,lat2,lon2)
        time=convert_time_reverse(datetime.datetime(year, month, day, hour, 0))
        slice_dictionary={longitudeparam:[lon1],latitudeparam:[lat1],timeparam:[int(time)]}
        currenttemp=ie.slice(edaparameter,slice_dictionary)
        val1=float(currenttemp)
        slice_dictionary={longitudeparam:[lon2],latitudeparam:[lat2],timeparam:[int(time)]}
        currenttemp=ie.slice(edaparameter,slice_dictionary)
        val2=float(currenttemp)
           
        idw1 = (val1/(dist1*dist1))+(val2/(dist2*dist2))
        idw2 = (1/(dist1*dist1))+(1/(dist2*dist2))
        idw = idw1/idw2
        finaleda = idw
    else:
        lat1 = listlonlat[0] 
        lat2 = listlonlat[0]
        lat3 = listlonlat[1]
        lat4 = listlonlat[1]
        lon1 = listlonlat[2]
        lon2 = listlonlat[3]
        lon3 = listlonlat[2]
        lon4 = listlonlat[3]
        dist1 = distancecalculation(latitude,longitude,lat1,lon1)
        dist2 = distancecalculation(latitude,longitude,lat2,lon2)
        dist3 = distancecalculation(latitude,longitude,lat3,lon3)
        dist4 = distancecalculation(latitude,longitude,lat4,lon4)
        time=convert_time_reverse(datetime.datetime(year, month, day,  hour, 0))
        slice_dictionary={longitudeparam:[lon1],latitudeparam:[lat1],timeparam:[int(time)]}
        currenttemp=ie.slice(edaparameter,slice_dictionary)
        val1=float(currenttemp)
        slice_dictionary={longitudeparam:[lon2],latitudeparam:[lat2],timeparam:[int(time)]}
        currenttemp=ie.slice(edaparameter,slice_dictionary)
        val2=float(currenttemp)
        slice_dictionary={longitudeparam:[lon3],latitudeparam:[lat3],timeparam:[int(time)]}
        currenttemp=ie.slice(edaparameter,slice_dictionary)
        val3=float(currenttemp)
        slice_dictionary={longitudeparam:[lon4],latitudeparam:[lat4],timeparam:[int(time)]}
        currenttemp=ie.slice(edaparameter,slice_dictionary)
        val4=float(currenttemp)
        
        idw1 = (val1/(dist1*dist1))+(val2/(dist2*dist2))+(val3/(dist3*dist3))+(val4/(dist4*dist4))
        idw2 = (1/(dist1*dist1))+(1/(dist2*dist2))+(1/(dist3*dist3))+(1/(dist4*dist4))
        idw = idw1/idw2
        finaleda = idw    
    return finaleda     

###  Function for one ERA point and one year:

In [10]:
from datetime import date, timedelta
def findfacteda(latitude,longitude,year,fnameyearly,fnameweekly, fnamedaily,exportfolder,frostdegree,unitcoeffsolar,unitcoeffwater,unitcoeffrunoff, dayinrow,starthourday,endhourday,hourstep,cropabsmax,cropabsmin,cropoptmax,cropoptmin,im,ie,tempparameter,eda_t,estarthourday,eendhourday,edastep,edatparam,edalonparam, edalatparam, edatimeparam,latparam, lonparam, timeparam,edalatlist,edalonlist,dprint, wprint, aprint, eda_s, eda_f, eda_w, calctemp, calcfert, calcwater, calcrunoff, calctempcrop, soilparameter, edafparam, fertdegree, dayinrowfert, solarparameter, edasparam, preciparameter, edapparam, evapoparameter, edaeparam, runoffparameter, edarparam):
     #!alg limitation - missing option frosthourday (how many hours below 0°C are considered as frostday) - not necessary
     #!alg specification - "frost day before" is calculated if part of interval (daymin-dminunc, daymin+dminunc) is under frost degree          
     #!alg. limitation - southern hemisphere not tested, daily print start in autumn half 1.7. -> 30.6., week during 30.6.-1.7. is split
        
    # temperature parameter:
    if calctemp == 1 : # if we work with temperature
        numbfrostdays=0 # for calculating number of frost days
        numbfrostdaysdown=0 # difference between calculation frost days using temperature value and (temperature value+eda)
        numbfrostdaysup=0 # difference between calculation frost days using temperature value and (temperature value-eda)
        lastfrostday=0 # last frost date
        lastfrostdayearlier=0 # last frost date - using temperature value + eda (yearlier than lastfrostday or same date)
        lastfrostdaylater=0 # last frost date - using temperature value - eda (later than lastfrostday or same date)
        daysbefore=0 #repetition of frost days in row - if day before is frost day -> daysbefore += 1, else = 0
        firstfrostday=0 # first frost date
        firstfrostdaylater=0 # first frost date - using temperature value + eda
        firstfrostdayearlier=0 # first frost date - using temperature value - eda
        cutfrost=0 #auxiliary variable for finding first frost date
        cutfrost1=0 #auxiliary variable for finding "first frost day later"
        cutfrost2=0 #auxiliary variable for finding "first frost day earlier"
        if calctempcrop == 1: # if we work with temperature and crop temperature thresholds 
            numbofgrowday=0 # for calculation number of growing day
            numbofgrowdaydown=0 # difference between temperature value and eda value - only part of eda interval is out of growing temperatures
            numbofgrowdayup=0 # difference between temperature value and eda value - only part of eda interval is in growing temperatures
            numbofoptgrowday=0 # for calculation number of optimal growing day
            numbofoptgrowdaydown=0 # difference between temperature value and eda value - only part of eda interval is out of optimal growing temperatures
            numbofoptgrowdayup=0 # difference between temperature value and eda value - only part of eda interval is in optimal growing temperatures
            wgdu = 0 #weekly growing degree units
            wgduuncdown = 0 # minimal weekly gdu (calculating with uncertainty)
            wgduuncup = 0 # maximal weekly gdu (calculating with uncertainty)
            whsu= 0 #weekly heat stress units
            whsuuncdown = 0
            whsuuncup = 0
            agdu = 0 #annual growing degree units
            agduuncdown = 0
            agduuncup = 0
            ahsu= 0 #annual heat stress unit
            ahsuuncdown = 0
            ahsuuncup = 0        
    # fertilize parameter:
    if calcfert == 1 :  # if we work with soil temperature
        lastfertday=0  #last date with soil temperature above X degree
        lastfertdaylater=0 # calculation of date of fertilization using soil temperature value + eda
        lastfertdayearlier=0 # calculation of date of fertilization using soil temperature value - eda
        daysbeforefert=0 #how many days in row have soil temperature under X degree
    # solar parameter:
    if calcsolar == 1 : # if we work with solar radiation
        wsolar = 0  # weekly solar radiation
        wsolarunc = 0 # uncertainty of weekly solar radiation
        asolar = 0 # annual solar radiation
        asolarunc = 0        
    # water parameter:
    if calcwater == 1 :  # if we work with precipitation, evapotranspiration, runoff
        wpreci = 0 # weekly precipitation
        wpreciunc = 0 #uncertainty of weekly precipitation
        apreci = 0 # annual precipitation
        apreciunc = 0
        wevapo = 0 # weekly evapotranspiration
        wevapounc = 0
        aevapo = 0 # annual evapotranspiration
        aevapounc = 0
        wrunoff = 0 # runoff
        wrunoffunc = 0
        arunoff = 0
        arunoffunc = 0
        abalanc = 0 # annual water balance
        abalancunc = 0            
    
    #determination of winter and summer: (southern/northern hemisphere)
    wintermonth=1 
    summermonth=7
    if latitude<0: #southern hemisphere
        wintermonth=7
        summermonth=1      
    #detection of week:
    week = 1 # order of the week in year
    dayofweek = 1 # order of the day in week
    
    # 1. Last spring frost date (half of year):
    startmonth=wintermonth
    endmonth=summermonth
    startdate=1
    enddate=1    
    if endmonth == 1: #southern hemisphere
        endmonth=12
        enddate=31
        firstdayyear = date(year, 1, 1)
        halfdayyear = date(year, 7, 1)
        auxweek = edate - sdate
        week = int(auxweek.days/7) #! alg. limitation - not tested
        dayofweek = (auxweek.days - (week*7)) #! alg. limitation - not tested
        weekhalfsouthernhemi = week #! alg. limitation - spliting of this week in weekly sums 
        
    sdate = date(year, startmonth, startdate)   # start date for searching last frost date
    edate = date(year, endmonth, enddate)   # end date for searching last frost date
    delta = edate - sdate       # as timedelta
    
    for i in range(delta.days):
        daylong = sdate + timedelta(days=i)
        sdaylong = str(daylong)
        tday = int(sdaylong[8:10])
        tmonth = int(sdaylong[5:7])
        tyear = int(sdaylong[0:4])        
        # temperature parameter:
        if calctemp == 1 :  # if we work with temperature
            daymin = 100 # minimal daily temperature
            dayminhour = 0 #hour of minimal daily temperature
            daymax = (-100)
            daymaxhour = 0
            dayavg = 0
            daytlist = [] #list for daily hourly temperatures
            eda2daytlist = [] #list for daily temperature uncertainties
            dminunc = 0.0 # uncertainty of minimal temperature
            dmaxunc = 0.0
            auxdminunc = 0.0 #auxiliary
            auxdmaxunc = 0.0 #auxiliary       
            davgunc = 0.0 # uncertainty of daily average temperature
            auxdavgunc = 0.0 #auxiliary
            if calctempcrop == 1 :  # if we calculate gdu hsu
                dhsu = 0 #daily heat stress unit
                dhsumax = 0 # maximal daily heat stress unit (calculation with uncertainty)
                dhsumin = 0 # minimal daily heat stress unit (calculation with uncertainty)
                dgdu = 0 # daily growing degree units
                dgdumax = 0
                dgdumin = 0            
        # solar parameter:
        if calcsolar == 1 :  # if we work with solar radiation
            dsolar = 0 # daily sum of solar radiation            
            dayslist = []
            eda2dayslist = []
            dsolarunc = 0 # daily eda sum of solar radiation (calculation with uncertainty)
        # water parameter:
        if calcwater == 1 :  # if we work with precipitation, evapotranspiration, runoff
            dpreci = 0 # daily sum of precipitation             
            devapo = 0 # daily sum of evapotranspiration             
            drunoff = 0 # daily sum of runoff            
            dayplist = []
            eda2dayplist = []
            dpreciunc = 0 # daily sum of precipitation uncertainties
            dayelist = []
            eda2dayelist = []
            devapounc = 0 # daily sum of evapotranspiration uncertainties
            dayrlist = [] 
            eda2dayrlist = []
            drunoffunc = 0 # daily sum of runoff uncertainties       
        
        for hour in range(starthourday, endhourday+1, 1): # for specific hours (all day,only sunrise hours,..)
            time=convert_time_reverse(datetime.datetime(tyear, tmonth, tday, hour, 0))
            slice_dictionary={lonparam:[longitude,],latparam:[latitude],timeparam:[int(time)]}           
            # temperature parameter:
            if calctemp == 1 :  # if we work with temperature
                currentparam=kelvin_to_celsius_vector(im.slice(tempparameter,slice_dictionary)) # reading netcdf temperatures
                current=round(float(currentparam), 2)
                daytlist.append(current)
                if current < daymin:
                    daymin=current
                    dayminhour=hour
                if current > daymax:
                    daymax=current
                    daymaxhour=hour
            # solar parameter:
            if calcsolar == 1 :  # if we work with solar radiation
                currentparam=im.slice(solarparameter,slice_dictionary)*unitcoeffsolar  
                current=round(float(currentparam), 2)
                dayslist.append(current)
            # water parameter:
            if calcwater == 1 :  # if we work with precipitation, evapotranspiration, runoff
                currentparam=im.slice(preciparameter,slice_dictionary)*unitcoeffwater 
                current=round(float(currentparam), 2)
                dayplist.append(current)
                currentparam= im.slice(evapoparameter,slice_dictionary)*unitcoeffwater 
                current=round(float(currentparam), 2)
                dayelist.append(current)
                if calcrunoff == 1 : # if we work with runoff
                    currentparam=im.slice(runoffparameter,slice_dictionary)*unitcoeffrunoff 
                    current=round(float(currentparam), 2)
                    dayrlist.append(current)           
        
        # temperature parameter:
        if calctemp == 1 :  # if we work with temperature
            auxdayavg = sum(daytlist)/len(daytlist)
            dayavg = round(auxdayavg, 2)        
        # solar parameter:
        if calcsolar == 1 :  # if we work with solar radiation
            auxdsolar = (sum(dayslist)/len(dayslist))*24
            dsolar = round(auxdsolar, 2)
        # water parameter:
        if calcwater == 1 :  # if we work with precipitation, evapotranspiration, runoff
            auxdpreci = (sum(dayplist)/len(dayplist))*24
            dpreci = round(auxdpreci, 2)
            auxdevapo = (sum(dayelist)/len(dayelist))*24*(-1)
            devapo = round(auxdevapo, 2)
            if calcrunoff == 1:
                auxdrunoff = (sum(dayrlist)/len(dayrlist))*24
                drunoff = round(auxdrunoff, 2)
            
        if eda_t!=0 or eda_s != 0 or eda_w != 0: #if we want to calculate edo for t-temperature, s-solar radiation, w-water balance                           
            for hour in range(estarthourday, eendhourday+1, edastep): # get hourly uncertainties
                # temperature parameter:
                if calctemp == 1 and eda_t!=0:  # if we work with temperature
                    edahoura = edahourvalue(latitude, longitude,edalatlist,edalonlist,ie,edatparam,edalonparam, edalatparam, edatimeparam, tyear, tmonth, tday, hour)
                    edahour = round(edahoura, 2) 
                    eda2daytlist.append(edahour*edahour)
                # solar parameter:
                if calcsolar == 1 and eda_s != 0:  # if we work with solar radiation
                    edahoura = edahourvalue(latitude, longitude,edalatlist,edalonlist,ie,edasparam,edalonparam, edalatparam, edatimeparam, tyear, tmonth, tday, hour)*unitcoeffsolar 
                    edahour = round(edahoura, 2) 
                    eda2dayslist.append(edahour*edahour)
                # water parameter:
                if calcwater == 1 and eda_w != 0:  # if we work with precipitation, evapotranspiration, runoff
                    edahoura = edahourvalue(latitude, longitude,edalatlist,edalonlist,ie,edapparam,edalonparam, edalatparam, edatimeparam, tyear, tmonth, tday, hour)*unitcoeffwater
                    edahour = round(edahoura, 2) 
                    eda2dayplist.append(edahour*edahour)
                    edahoura = edahourvalue(latitude, longitude,edalatlist,edalonlist,ie,edaeparam,edalonparam, edalatparam, edatimeparam, tyear, tmonth, tday, hour)*unitcoeffwater
                    edahour = round(edahoura, 2) 
                    eda2dayelist.append(edahour*edahour)
                    if calcrunoff == 1:
                        edahoura = edahourvalue(latitude, longitude,edalatlist,edalonlist,ie,edarparam,edalonparam, edalatparam, edatimeparam, tyear, tmonth, tday, hour)*unitcoeffrunoff
                        edahour = round(edahoura, 2) 
                        eda2dayrlist.append(edahour*edahour)                        
            
            # temperature parameter:
            if calctemp == 1 and eda_t!=0:  # if we work with temperature       
                 # eda day average: 
                auxdavgunc = math.sqrt(sum(eda2daytlist)/len(eda2daytlist))  
                davgunc = round(auxdavgunc, 2) 
                # eda day min: 
                tval = int(dayminhour/edastep)
                if (dayminhour % edastep) == 0:
                    auxdminunc = math.sqrt(eda2daytlist[tval])
                elif dayminhour > (24-edastep): #!alg limitation - interpolation between 21 and 00 neglected
                    auxdminunc = math.sqrt(eda2daytlist[-1])
                else:
                        tval1 = edastep*tval
                        tval2 = tval1+edastep
                        tval3 = dayminhour - tval1
                        tval4 = tval2 - dayminhour 
                        tval5 = tval3+tval4
                        tval31 = tval3/tval5 #1/3 or 2/3 if eda step==3h
                        tval41 = tval4/tval5 #1/3 or 2/3 if eda step==3h
                        auxdminunc = tval41*math.sqrt(eda2daytlist[tval]) +  tval31*math.sqrt(eda2daytlist[tval+1])
                dminunc = round(auxdminunc, 2)       
                 # eda day max:
                tval = int(daymaxhour/edastep)
                if (daymaxhour % edastep) == 0:
                    auxdmaxunc =  math.sqrt(eda2daytlist[tval]) 
                elif daymaxhour > (24-edastep): #!alg limitation - interpolation between 21 and 00 neglected
                    auxdmaxunc =  math.sqrt(eda2daytlist[-1])
                else:
                        tval1 = edastep*tval
                        tval2 = tval1+edastep
                        tval3 = daymaxhour - tval1
                        tval4 = tval2 - daymaxhour 
                        tval5 = tval3+tval4
                        tval31 = tval3/tval5
                        tval41 = tval4/tval5
                        auxdmaxunc =  (tval31*math.sqrt(eda2daytlist[tval]) + tval41*math.sqrt(eda2daytlist[tval+1]))
                dmaxunc = round(auxdmaxunc, 2)
            # solar parameter:
            if calcsolar == 1 and eda_s != 0:  # if we work with solar radiation
                # eda day solar sum: 
                dsolarunc1 = math.sqrt(sum(eda2dayslist)/len(eda2dayslist))*24  
                dsolarunc = round(dsolarunc1, 2) 
            # water parameter:
            if calcwater == 1 and eda_w != 0:  # if we work with precipitation, evapotranspiration, runoff
                # eda day preci, evapo, runoff sum: 
                dpreciunc1 = math.sqrt(sum(eda2dayplist)/len(eda2dayplist))*24  
                dpreciunc = round(dpreciunc1, 2) 
                devapounc1 = math.sqrt(sum(eda2dayelist)/len(eda2dayelist))*24  
                devapounc = round(devapounc1, 2) 
                if calcrunoff == 1:
                    drunoffunc1 = math.sqrt(sum(eda2dayrlist)/len(eda2dayrlist))*24  
                    drunoffunc = round(drunoffunc1, 2) 
                 
        # temperature parameter:
        if calctemp == 1 :  # if we work with temperature
            if calctempcrop == 1 :
                if (dayavg+davgunc) >= cropabsmin and (dayavg-davgunc) <= cropabsmax:  #growing day?
                    if dayavg >= cropabsmin and dayavg <= cropabsmax:
                        numbofgrowday+=1
                        if (dayavg-davgunc) < cropabsmin or (dayavg+davgunc) > cropabsmax:
                            numbofgrowdaydown+=1
                    else:
                        numbofgrowdayup+=1
                if (dayavg+davgunc) >= cropoptmin and (dayavg-davgunc) <= cropoptmax: #optimal growing day?
                    if dayavg >= cropoptmin and dayavg <= cropoptmax:
                        numbofoptgrowday+=1
                        if (dayavg-davgunc) < cropoptmin or (dayavg+davgunc) > cropoptmax:
                            numbofoptgrowdaydown+=1
                    else:
                        numbofoptgrowdayup+=1            
                #gdu:
                #1.gdu
                auxt = dayavg        
                if auxt >= cropabsmin and auxt <= cropabsmax:  
                    dgdu = round((auxt-cropabsmin), 2)
                #2.gdu+eda
                auxt = dayavg+davgunc        
                if auxt >= cropabsmin and auxt <= cropabsmax:  # !alg limitation - possible problem for very warm areas where the avg temp is higher than cropmax -> dgdumax does not have to be max - neglected 
                    dgdumax = auxt-cropabsmin
                #3.gdu-eda
                auxt = dayavg-davgunc        
                if auxt >= cropabsmin and auxt <= cropabsmax: # !alg limitation - possible problem for very warm areas where the avg temp is higher than cropmax -> dgdumin does not have to be min  - neglected  
                    dgdumin = auxt-cropabsmin
                #3.gdu unc
                dgduuncup = round((abs(dgdu-dgdumax)), 2)
                dgduuncdown = round((abs(dgdu-dgdumin)), 2)
                #hsu:
                #1.hsu
                auxt = daymax        
                if auxt > cropabsmax:  
                    dhsu = round((auxt-cropabsmax), 2)
                #2.hsu+eda
                auxt = daymax+dmaxunc        
                if auxt > cropabsmax: 
                    dhsumax = auxt-cropabsmax
                #3.hsu-eda
                auxt = daymax-dmaxunc        
                if auxt > cropabsmax:  
                    dhsumin = auxt-cropabsmax
                #hsu unc
                dhsuuncup = round((abs(dhsu-dhsumax)), 2)
                dhsuuncdown = round((abs(dhsu-dhsumin)), 2)

                wgdu += dgdu
                wgduuncdown += dgduuncdown
                wgduuncup += dgduuncup
                whsu += dhsu
                whsuuncdown += dhsuuncdown
                whsuuncup += dhsuuncup
                agdu += dgdu
                agduuncdown += dgduuncdown
                agduuncup += dgduuncup
                ahsu += dhsu
                ahsuuncdown += dhsuuncdown
                ahsuuncup += dhsuuncup   
           
        #----------------------------------------------printing csv daily--------------------------------- (
        if dprint == 1:
            ttext= str(latitude) + ", " + str(longitude) + ", " 
            ttext = ttext + str(tday) + "." + str(tmonth) + "." + str(tyear) 
            if calctemp == 1: 
                ttext= ttext + ", " +  str(dayavg) + ", " + str(davgunc) + ", " + str(daymin) + ", " + str(dminunc) + ", " + str(daymax) + ", " + str(dmaxunc)
                            
            print_csv(exportfolder,fnamedaily, ttext, 0)            
        #----------------------------------------------printing csv daily--------------------------------- ) 
        
        # temperature parameter:
        if calctemp == 1 :  # if we work with temperature
            if daymin<=frostdegree and (daymin+dminunc)<=frostdegree:  # frostday?
                numbfrostdays+=1
                if daysbefore>=dayinrow-1:
                    lastfrostday=daylong
                    lastfrostdayearlier=daylong
                    lastfrostdaylater=daylong
                daysbefore+=1
            elif daymin<=frostdegree and (daymin+dminunc)>frostdegree:
                numbfrostdays+=1
                numbfrostdaysdown+=1
                if daysbefore>=dayinrow-1:
                    lastfrostday=daylong
                    lastfrostdaylater=daylong
                daysbefore+=1
            elif daymin>frostdegree and (daymin-dminunc)<=frostdegree:
                numbfrostdaysup+=1
                if daysbefore>=dayinrow-1:
                    lastfrostdaylater=daylong
                daysbefore+=1
            else:
                daysbefore=0
        # solar parameter:
        if calcsolar == 1 :  # if we work with solar radiation
            wsolar += dsolar
            wsolarunc += dsolarunc
            asolar += dsolar
            asolarunc += dsolarunc            
        # water parameter:
        if calcwater == 1 :  # if we work with precipitation, evapotranspiration, runoff
            wpreci += dpreci
            wpreciunc += dpreciunc
            apreci += dpreci
            apreciunc += dpreciunc    
            wevapo += devapo
            wevapounc += devapounc
            aevapo += devapo
            aevapounc += devapounc
            if calcrunoff ==1:               
                wrunoff += drunoff
                wrunoffunc += drunoffunc               
                arunoff += drunoff
                arunoffunc += drunoffunc 
    
        if dayofweek == 7:                       
            #----------------------------------------------printing csv weekly--------------------------------- (
            if wprint == 1:
                ttext= str(latitude) + ", " + str(longitude) + ", " 
                ttext = ttext + str(tyear) +  ", " + str(week)
                if calctempcrop == 1:
                    ttext = ttext + ", " +  str(round(wgdu,2)) + ", " + str(round(wgduuncup,2)) + ", " + str(round(wgduuncdown,2)) + ", " + str(round(whsu,2)) + ", " + str(round(whsuuncup,2)) + ", " + str(round(whsuuncdown,2))
                if calcsolar == 1:
                    ttext = ttext + ", " +  str(round(wsolar,2)) + ", " + str(round(wsolarunc,2))  
                if calcwater == 1:
                    ttext = ttext + ", " +  str(round(wpreci,2)) + ", " + str(round(wpreciunc,2)) + ", " +  str(round(wevapo,2)) + ", " + str(round(wevapounc,2))  
                if calcrunoff == 1: 
                    ttext = ttext + ", " +  str(round(wrunoff,2)) + ", " + str(round(wrunoffunc,2))                     

                print_csv(exportfolder,fnameweekly, ttext, 0)
            #----------------------------------------------printing csv weekly--------------------------------- )
            # temperature parameter:
            if calctempcrop == 1 :  # if we work with gdu,hsu                       
                wgdu = 0
                wgduuncdown = 0
                wgduuncup = 0
                whsu = 0
                whsuuncdown = 0
                whsuuncup = 0
            # solar parameter:
            if calcsolar == 1 :  # if we work with solar radiation                
                wsolar = 0
                wsolarunc = 0
            # water parameter:
            if calcwater == 1 :  # if we work with precipitation, evapotranspiration, runoff
                wpreci = 0
                wpreciunc = 0                              
                wevapo = 0
                wevapounc = 0                
                wrunoff = 0
                wrunoffunc = 0                          
                                       
            week += 1
            dayofweek = 1
        else:
            if latitude<0: #southern hemisphere (last week in year during this spring period)
                if daylong==edate:                    
           #----------------------------------------------printing csv weekly--------------------------------- (
                    if wprint == 1:
                        ttext= str(latitude) + ", " + str(longitude) + ", " 
                        ttext = ttext + str(tyear) +  ", " + str(week)
                        if calctempcrop == 1:
                            ttext = ttext + ", " +  str(round(wgdu,2)) + ", " + str(round(wgduuncup,2)) + ", " + str(round(wgduuncdown,2)) + ", " + str(round(whsu,2)) + ", " + str(round(whsuuncup,2)) + ", " + str(round(whsuuncdown,2))
                        if calcsolar == 1:
                            ttext = ttext + ", " +  str(round(wsolar,2)) + ", " + str(round(wsolarunc,2))  
                        if calcwater == 1:
                            ttext = ttext + ", " +  str(round(wpreci,2)) + ", " + str(round(wpreciunc,2)) + ", " +  str(round(wevapo,2)) + ", " + str(round(wevapounc,2))  
                        if calcrunoff == 1: 
                            ttext = ttext + ", " +  str(round(wrunoff,2)) + ", " + str(round(wrunoffunc,2))                     

                        print_csv(exportfolder,fnameweekly, ttext, 0)
            #----------------------------------------------printing csv weekly--------------------------------- )
                
            dayofweek += 1          
    #--------------------------------------------------------------------------------------------------------------------------------------------------------------         
    #-------------------------------------------------------------------------------------------------------------------------------------------------------------- 
    # 2. First autumn frost date (half of year):
    startmonth=summermonth
    endmonth=wintermonth
    enddate=1
    startdate=1
    if endmonth == 1:
        endmonth=12
        enddate=31
    else: # southern hemisphere
        week = 1 
        dayofweek = 1 
        
    sdate = date(year, startmonth, startdate)   # start date of searching
    edate = date(year, endmonth, enddate)   # end date of searching
    delta = edate - sdate      # as timedelta
    
    # temperature parameters:
    if calctemp == 1 :  # if we work with temperature
        daysbefore=0
        
    for i in range(delta.days+1):
        daylong = sdate + timedelta(days=i)
        sdaylong = str(daylong)
        tday = int(sdaylong[8:10])
        tmonth = int(sdaylong[5:7])
        tyear = int(sdaylong[0:4])        
        # temperature parameter:
        if calctemp == 1 :  # if we work with temperature
            daymin = 100
            dayminhour = 0
            daymax = (-100)
            daymaxhour = 0
            dayavg = 0
            daytlist = []
            eda2daytlist = []
            dminunc = 0.0
            dmaxunc = 0.0
            auxdminunc = 0.0
            auxdmaxunc = 0.0                        
            davgunc = 0.0
            auxdavgunc = 0.0
            if calctempcrop == 1 :
                dhsu = 0
                dhsumax = 0
                dhsumin = 0            
                dgdu = 0
                dgdumax = 0
                dgdumin = 0            
        # fertilize parameter:
        if calcfert == 1 :  # if we work with soil temperature
            daysavg = 0
            dayflist = []
            eda2dayflist = []
            dsavgunc = 0.0
            auxdsavgunc = 0.0            
        # solar parameter:
        if calcsolar == 1 :  # if we work with solar radiation
            dsolar = 0            
            dayslist = []
            eda2dayslist = []
            dsolarunc = 0
         # water parameter:
        if calcwater == 1 :  # if we work with precipitation, evapotranspiration, runoff
            dpreci = 0 # daily sum of precipitation            
            devapo = 0 # daily sum of evapotranspiration            
            drunoff = 0 # daily sum of runoff           
            dayplist = []
            eda2dayplist = []
            dpreciunc = 0  # daily eda sum of precipitation
            dayelist = []
            eda2dayelist = []
            devapounc = 0
            dayrlist = []
            eda2dayrlist = []
            drunoffunc = 0
            
        for hour in range(starthourday, endhourday+1, 1): # for specific hours (all day,only sunrise hours,..)
            time=convert_time_reverse(datetime.datetime(tyear, tmonth, tday, hour, 0))
            slice_dictionary={lonparam:[longitude,],latparam:[latitude],timeparam:[int(time)]}
            # temperature parameter:
            if calctemp == 1 :  # if we work with temperature
                currentparam=kelvin_to_celsius_vector(im.slice(tempparameter,slice_dictionary))
                current=float(currentparam)
                daytlist.append(current)
                if current < daymin:
                    daymin=current
                    dayminhour=hour
                if current > daymax:
                    daymax=current
                    daymaxhour=hour
            # fertilize parameter:
            if calcfert == 1 :  # if we work with soil temperature
                currentparam=kelvin_to_celsius_vector(im.slice(soilparameter,slice_dictionary))
                current=float(currentparam)
                dayflist.append(current)
            # solar parameter:
            if calcsolar == 1 :  # if we work with solar radiation
                currentparam=im.slice(solarparameter,slice_dictionary)*unitcoeffsolar 
                current=round(float(currentparam), 2)
                dayslist.append(current)
            # water parameter:
            if calcwater == 1 :  # if we work with precipitation, evapotranspiration, runoff
                currentparam=im.slice(preciparameter,slice_dictionary)*unitcoeffwater
                current=round(float(currentparam), 2)
                dayplist.append(current)
                currentparam=im.slice(evapoparameter,slice_dictionary)*unitcoeffwater
                current=round(float(currentparam), 2)
                dayelist.append(current)
                if calcrunoff == 1 :
                    currentparam=im.slice(runoffparameter,slice_dictionary)*unitcoeffrunoff
                    current=round(float(currentparam), 2)
                    dayrlist.append(current)
        # temperature parameter:
        if calctemp == 1 :  # if we work with temperature            
            auxdayavg = sum(daytlist)/len(daytlist)
            dayavg = round(auxdayavg, 2)
         # fertilize parameter:
        if calcfert == 1 :  # if we work with soil temperature
            daysavga = sum(dayflist)/len(dayflist)
            daysavg = round(daysavga, 2)        
        # solar parameter:
        if calcsolar == 1 :  # if we work with solar radiation
            auxdsolar = (sum(dayslist)/len(dayslist))*24
            dsolar = round(auxdsolar, 2)
        # water parameter:
        if calcwater == 1 :  # if we work with precipitation, evapotranspiration, runoff
            auxdpreci = (sum(dayplist)/len(dayplist))*24
            dpreci = round(auxdpreci, 2)
            auxdevapo = (sum(dayelist)/len(dayelist))*24*(-1)
            devapo = round(auxdevapo, 2)
            if calcrunoff == 1:
                auxdrunoff = (sum(dayrlist)/len(dayrlist))*24
                drunoff = round(auxdrunoff, 2)
            
        if eda_t!=0 or eda_s != 0 or eda_w != 0 or eda_f !=0 :
                           
            for hour in range(estarthourday, eendhourday+1, edastep): # get hourly uncertainties
                # temperature parameter:
                if calctemp == 1 and eda_t!=0:  # if we work with temperature      
                    edahoura = edahourvalue(latitude, longitude,edalatlist,edalonlist,ie,edatparam,edalonparam, edalatparam, edatimeparam, tyear, tmonth, tday, hour)
                    edahour = round(edahoura, 2) 
                    eda2daytlist.append(edahour*edahour)
                 # fertilize parameter:
                if calcfert == 1 and eda_f!=0:  # if we work with soil temperature
                    edahoura = edahourvalue(latitude, longitude,edalatlist,edalonlist,ie,edafparam,edalonparam, edalatparam, edatimeparam, tyear, tmonth, tday, hour)
                    edahour = round(edahoura, 2) 
                    eda2dayflist.append(edahour*edahour)
                # solar parameter:
                if calcsolar == 1 and eda_s!=0:  # if we work with solar radiation
                    edahoura = edahourvalue(latitude, longitude,edalatlist,edalonlist,ie,edasparam,edalonparam, edalatparam, edatimeparam, tyear, tmonth, tday, hour)*unitcoeffsolar 
                    edahour = round(edahoura, 2) 
                    eda2dayslist.append(edahour*edahour)
                # water parameter:
                if calcwater == 1 and eda_w!=0:  # if we work with precipitation, evapotranspiration, runoff
                    edahoura = edahourvalue(latitude, longitude,edalatlist,edalonlist,ie,edapparam,edalonparam, edalatparam, edatimeparam, tyear, tmonth, tday, hour)*unitcoeffwater
                    edahour = round(edahoura, 2) 
                    eda2dayplist.append(edahour*edahour)
                    edahoura = edahourvalue(latitude, longitude,edalatlist,edalonlist,ie,edaeparam,edalonparam, edalatparam, edatimeparam, tyear, tmonth, tday, hour)*unitcoeffwater
                    edahour = round(edahoura, 2) 
                    eda2dayelist.append(edahour*edahour)
                    if calcrunoff == 1:
                        edahoura = edahourvalue(latitude, longitude,edalatlist,edalonlist,ie,edarparam,edalonparam, edalatparam, edatimeparam, tyear, tmonth, tday, hour)*unitcoeffrunoff
                        edahour = round(edahoura, 2) 
                        eda2dayrlist.append(edahour*edahour)    
                    
            # temperature parameter:
            if calctemp == 1 :  # if we work with temperature         
                # eda day average: 
                auxdavgunc = math.sqrt(sum(eda2daytlist)/len(eda2daytlist))  
                davgunc = round(auxdavgunc, 2) 
                # eda day min: 
                tval = int(dayminhour/edastep)
                if (dayminhour % edastep) == 0:
                    auxdminunc = math.sqrt(eda2daytlist[tval])
                elif dayminhour > (24-edastep): #!alg limitation - interpolation between 21 and 00 neglected
                    auxdminunc = math.sqrt(eda2daytlist[-1])
                else:
                        tval1 = edastep*tval
                        tval2 = tval1+edastep
                        tval3 = dayminhour - tval1
                        tval4 = tval2 - dayminhour 
                        tval5 = tval3+tval4
                        tval31 = tval3/tval5 #1/3 or 2/3 if eda step==3h
                        tval41 = tval4/tval5 #1/3 or 2/3 if eda step==3h
                        auxdminunc = tval41*math.sqrt(eda2daytlist[tval]) +  tval31*math.sqrt(eda2daytlist[tval+1])
                dminunc = round(auxdminunc, 2)       
                 # eda day max:
                tval = int(daymaxhour/edastep)
                if (daymaxhour % edastep) == 0:
                    auxdmaxunc =  math.sqrt(eda2daytlist[tval]) 
                elif daymaxhour > (24-edastep): #!alg limitation - interpolation between 21 and 00 neglected
                    auxdmaxunc =  math.sqrt(eda2daytlist[-1])  
                else:
                        tval1 = edastep*tval
                        tval2 = tval1+edastep
                        tval3 = daymaxhour - tval1
                        tval4 = tval2 - daymaxhour 
                        tval5 = tval3+tval4
                        tval31 = tval3/tval5
                        tval41 = tval4/tval5
                        auxdmaxunc =  (tval31*math.sqrt(eda2daytlist[tval]) +  tval41*math.sqrt(eda2daytlist[tval+1]))

                dmaxunc = round(auxdmaxunc, 2)  
            # fertilize parameter:
            if calcfert == 1 :  # if we work with soil temperature
                # eda day soil average: 
                auxdsavgunc = math.sqrt(sum(eda2dayflist)/len(eda2dayflist))  
                dsavgunc = round(auxdsavgunc, 2) 
            # solar parameter:
            if calcsolar == 1 :  # if we work with solar radiation
                 # eda day solar sum: 
                dsolarunc1 = math.sqrt(sum(eda2dayslist)/len(eda2dayslist))*24  
                dsolarunc = round(dsolarunc1, 2) 
            # water parameter:
            if calcwater == 1 :  # if we work with precipitation, evapotranspiration, runoff
                # eda day preci, evapo, runoff sum: 
                dpreciunc1 = math.sqrt(sum(eda2dayplist)/len(eda2dayplist))*24  
                dpreciunc = round(dpreciunc1, 2) 
                devapounc1 = math.sqrt(sum(eda2dayelist)/len(eda2dayelist))*24  
                devapounc = round(devapounc1, 2) 
                if calcrunoff == 1:
                    drunoffunc1 = math.sqrt(sum(eda2dayrlist)/len(eda2dayrlist))*24  
                    drunoffunc = round(drunoffunc1, 2)                 
        
        # temperature parameter:
        if calctemp == 1 :  # if we work with temperature 
            if calctempcrop == 1 :
                if (dayavg+davgunc) >= cropabsmin and (dayavg-davgunc) <= cropabsmax:  #growing day?
                    if dayavg >= cropabsmin and dayavg <= cropabsmax:
                        numbofgrowday+=1
                        if (dayavg-davgunc) < cropabsmin or (dayavg+davgunc) > cropabsmax:
                            numbofgrowdaydown+=1
                    else:
                        numbofgrowdayup+=1
                if (dayavg+davgunc) >= cropoptmin and (dayavg-davgunc) <= cropoptmax: #optimal growing day?
                    if dayavg >= cropoptmin and dayavg <= cropoptmax:
                        numbofoptgrowday+=1
                        if (dayavg-davgunc) < cropoptmin or (dayavg+davgunc) > cropoptmax:
                            numbofoptgrowdaydown+=1
                    else:
                        numbofoptgrowdayup+=1            
                #gdu:
                #1.gdu
                auxt = dayavg        
                if auxt >= cropabsmin and auxt <= cropabsmax:  
                    dgdu = round((auxt-cropabsmin), 2)
                #2.gdu+eda
                auxt = dayavg+davgunc        
                if auxt >= cropabsmin and auxt <= cropabsmax:  
                    dgdumax = auxt-cropabsmin
                #3.gdu-eda
                auxt = dayavg-davgunc        
                if auxt >= cropabsmin and auxt <= cropabsmax:  
                    dgdumin = auxt-cropabsmin
                #3.gdu unc
                dgduuncup = round((abs(dgdu-dgdumax)), 2)
                dgduuncdown = round((abs(dgdu-dgdumin)), 2)
                #hsu:
                #1.hsu
                auxt = daymax        
                if auxt > cropabsmax:  
                    dhsu = round((auxt-cropabsmax), 2)
                #2.hsu+eda
                auxt = daymax+dmaxunc        
                if auxt > cropabsmax: 
                    dhsumax = auxt-cropabsmax
                #3.hsu-eda
                auxt = daymax-dmaxunc        
                if auxt > cropabsmax:  
                    dhsumin = auxt-cropabsmax
                #hsu unc
                dhsuuncup = round((abs(dhsu-dhsumax)), 2)
                dhsuuncdown = round((abs(dhsu-dhsumin)), 2)

                wgdu += dgdu
                wgduuncdown += dgduuncdown
                wgduuncup += dgduuncup
                whsu += dhsu
                whsuuncdown += dhsuuncdown
                whsuuncup += dhsuuncup
                agdu += dgdu
                agduuncdown += dgduuncdown
                agduuncup += dgduuncup
                ahsu += dhsu
                ahsuuncdown += dhsuuncdown
                ahsuuncup += dhsuuncup
                
        #----------------------------------------------printing csv daily--------------------------------- (
        if dprint == 1:
            ttext= str(latitude) + ", " + str(longitude) + ", " 
            ttext = ttext + str(tday) + "." + str(tmonth) + "." + str(tyear) 
            if calctemp == 1: 
                ttext= ttext + ", " +  str(dayavg) + ", " + str(davgunc) + ", " + str(daymin) + ", " + str(dminunc) + ", " + str(daymax) + ", " + str(dmaxunc)
            print_csv(exportfolder,fnamedaily, ttext, 0)
        #----------------------------------------------printing csv daily--------------------------------- )
        
        # temperature parameter:
        if calctemp == 1 :  # if we work with temperature 
            if daymin>frostdegree and (daymin-dminunc)<=frostdegree:  # frostday?
                numbfrostdaysup+=1
                if daysbefore>=(dayinrow-1) and cutfrost2==0:
                    firstfrostdayearlier=daylong
                    cutfrost2=1
                daysbefore+=1
            elif daymin<=frostdegree and (daymin+dminunc)>frostdegree:  # frostday?
                numbfrostdays+=1
                numbfrostdaysdown+=1
                if daysbefore>=(dayinrow-1) and cutfrost==0:
                    firstfrostday=daylong
                    cutfrost=1
                if daysbefore>=(dayinrow-1) and cutfrost2==0:
                    firstfrostdayearlier=daylong
                    cutfrost2=1
                daysbefore+=1
            elif daymin<=frostdegree and (daymin+dminunc)<=frostdegree:  # frostday?
                numbfrostdays+=1
                if daysbefore>=(dayinrow-1) and cutfrost==0:
                    firstfrostday=daylong
                    cutfrost=1
                if daysbefore>=(dayinrow-1) and cutfrost1==0:
                    firstfrostdaylater=daylong
                    cutfrost1=1
                if daysbefore>=(dayinrow-1) and cutfrost2==0:
                    firstfrostdayearlier=daylong
                    cutfrost2=1
                daysbefore+=1
            else:
                daysbefore=0
        # fertilize parameter:
        if calcfert == 1 :  # if we work with soil temperature
            if daysavg>=fertdegree and (daysavg-dsavgunc)>=fertdegree:  # fertday?                
                if daysbeforefert>=dayinrowfert-1:
                    lastfertday=daylong
                    lastfertdaylater=daylong
                    lastfertdayearlier=daylong
                daysbeforefert+=1
            elif daysavg>=fertdegree and (daysavg-dsavgunc)<fertdegree:                
                if daysbeforefert>=dayinrowfert-1:
                    lastfertday=daylong
                    lastfertdaylater=daylong 
                daysbeforefert+=1    
            elif daysavg<fertdegree and (daysavg+dsavgunc)>=fertdegree:
                if daysbeforefert>=dayinrowfert-1:
                    lastfertdaylater=daylong 
                daysbeforefert+=1
            else:
                daysbeforefert=0
        # solar parameter:
        if calcsolar == 1 :  # if we work with solar radiation
            wsolar += dsolar
            wsolarunc += dsolarunc
            asolar += dsolar
            asolarunc += dsolarunc   
        # water parameter:
        if calcwater == 1 :  # if we work with precipitation, evapotranspiration, runoff
            wpreci += dpreci
            wpreciunc += dpreciunc
            apreci += dpreci
            apreciunc += dpreciunc    
            wevapo += devapo
            wevapounc += devapounc
            aevapo += devapo
            aevapounc += devapounc
            if calcrunoff ==1:               
                wrunoff += drunoff
                wrunoffunc += drunoffunc               
                arunoff += drunoff
                arunoffunc += drunoffunc  
            
        if dayofweek == 7:            
           #----------------------------------------------printing csv weekly--------------------------------- (
            if wprint == 1:
                ttext= str(latitude) + ", " + str(longitude) + ", " 
                ttext = ttext + str(tyear) +  ", " + str(week)
                if calctempcrop == 1:
                    ttext = ttext + ", " +  str(round(wgdu,2)) + ", " + str(round(wgduuncup,2)) + ", " + str(round(wgduuncdown,2)) + ", " + str(round(whsu,2)) + ", " + str(round(whsuuncup,2)) + ", " + str(round(whsuuncdown,2))
                if calcsolar == 1:
                    ttext = ttext + ", " +  str(round(wsolar,2)) + ", " + str(round(wsolarunc,2))  
                if calcwater == 1:
                    ttext = ttext + ", " +  str(round(wpreci,2)) + ", " + str(round(wpreciunc,2)) + ", " +  str(round(wevapo,2)) + ", " + str(round(wevapounc,2))  
                if calcrunoff == 1: 
                    ttext = ttext + ", " +  str(round(wrunoff,2)) + ", " + str(round(wrunoffunc,2))                     

                print_csv(exportfolder,fnameweekly, ttext, 0)
            #----------------------------------------------printing csv weekly--------------------------------- )
            # temperature parameter:
            if calctempcrop == 1 :  # if we work with temperature                
                wgdu = 0
                wgduuncdown = 0
                wgduuncup = 0
                whsu = 0
                whsuuncdown = 0
                whsuuncup = 0
             # solar parameter:
            if calcsolar == 1 :  # if we work with solar radiation                
                wsolar = 0
                wsolarunc = 0               
            # water parameter:
            if calcwater == 1 :  # if we work with precipitation, evapotranspiration, runoff
                wpreci = 0
                wpreciunc = 0                              
                wevapo = 0
                wevapounc = 0
                wrunoff = 0
                wrunoffunc = 0                
            week += 1
            dayofweek = 1
        elif daylong==edate:
           #----------------------------------------------printing csv weekly--------------------------------- (
            if wprint == 1:
                ttext= str(latitude) + ", " + str(longitude) + ", " 
                ttext = ttext + str(tyear) +  ", " + str(week)
                if calctempcrop == 1:
                    ttext = ttext + ", " +  str(round(wgdu,2)) + ", " + str(round(wgduuncup,2)) + ", " + str(round(wgduuncdown,2)) + ", " + str(round(whsu,2)) + ", " + str(round(whsuuncup,2)) + ", " + str(round(whsuuncdown,2))
                if calcsolar == 1:
                    ttext = ttext + ", " +  str(round(wsolar,2)) + ", " + str(round(wsolarunc,2))  
                if calcwater == 1:
                    ttext = ttext + ", " +  str(round(wpreci,2)) + ", " + str(round(wpreciunc,2)) + ", " +  str(round(wevapo,2)) + ", " + str(round(wevapounc,2))  
                if calcrunoff == 1: 
                    ttext = ttext + ", " +  str(round(wrunoff,2)) + ", " + str(round(wrunoffunc,2))                     

                print_csv(exportfolder,fnameweekly, ttext, 0)
            #----------------------------------------------printing csv weekly--------------------------------- )
        else:
            dayofweek += 1
    
    aux1 = delta # auxiliary variable (same type as variable delta (date time))
    aux2 = delta # auxiliary variable (same type as variable delta (date time))
    # temperature parameter:
    if calctemp == 1 :  # if we work with temperature
        frostfreeperiod=0     
        frfreeperiodlonger=0
        frfreeperiodshorter=0
        firstyeardate = date(year, 1, 1)   # start date of year
        lastyeardate = date(year, 12, 31)   # end date of year         
        # Frostfreeperiod
        firstdayyear = date(year, 1, 1) # for calculation of number of days in year (normal/leap years)
        lastdayyear = date(year, 12, 31)
        auxyear = lastdayyear - firstdayyear
        yearlong = auxyear.days + 1
        if firstfrostday!=0 and lastfrostday!=0:
            if latitude>0:
                frostfreeperiod = firstfrostday-lastfrostday
                frostfreeperiod = (frostfreeperiod.days-1)  # -1 due to correct frost free period without threshold frost days
            else:                
                frostfreeperiod =  (firstfrostday-firstyeardate)+(lastyeardate-lastfrostday)
                frostfreeperiod = frostfreeperiod.days
        else:
            if firstfrostday!=0:
                if latitude>0:
                    frostfreeperiod = lastyeardate-firstfrostday
                    frostfreeperiod = yearlong-(frostfreeperiod.days+1) 
                else:                    
                    frostfreeperiod = firstfrostday-firstyeardate
                    frostfreeperiod = yearlong-(frostfreeperiod.days+1)
            elif lastfrostday!=0:
                if latitude>0:
                    frostfreeperiod = lastfrostday-firstyeardate
                    frostfreeperiod = yearlong-(frostfreeperiod.days+1)
                else:
                    frostfreeperiod = lastyeardate-lastfrostday
                    frostfreeperiod = yearlong-(frostfreeperiod.days+1)
            else:
                frostfreeperiod=yearlong 
        if firstfrostdaylater!=0 and lastfrostdayearlier!=0:
            if latitude>0:
                frfreeperiodlonger =  firstfrostdaylater-lastfrostdayearlier #usually more days than frostfreeperiod
                frfreeperiodlonger = (frfreeperiodlonger.days-1)
            else:
                firstyeardate = date(year, 1, 1)   # start date of year
                lastyeardate = date(year, 12, 31)   # end date of year
                frfreeperiodlonger =  (firstfrostdaylater-firstyeardate)+(lastyeardate1-lastfrostday)
                frfreeperiodlonger = frfreeperiodlonger.days
        else:
            if firstfrostdaylater!=0:
                if latitude>0:                     
                    frfreeperiodlonger = lastyeardate-firstfrostdaylater
                    frfreeperiodlonger = yearlong-(frfreeperiodlonger.days+1) 
                else:
                    frfreeperiodlonger = firstfrostdaylater-firstyeardate
                    frfreeperiodlonger = yearlong-(frfreeperiodlonger.days+1)                    
            elif lastfrostdayearlier!=0:
                if latitude>0:
                    frfreeperiodlonger = lastfrostdayearlier-firstyeardate
                    frfreeperiodlonger = yearlong-(frfreeperiodlonger.days+1)                     
                else:
                    frfreeperiodlonger = lastyeardate-lastfrostdayearlier
                    frfreeperiodlonger = yearlong-(frfreeperiodlonger.days+1)                     
            else:
                frfreeperiodlonger=yearlong          
        if firstfrostdayearlier!=0 and lastfrostdaylater!=0:
            if latitude>0:
                frfreeperiodshorter =  firstfrostdayearlier-lastfrostdaylater
                frfreeperiodshorter = (frfreeperiodshorter.days-1) #usually fewer days than frostfreeperiod
            else:                    
                frfreeperiodshorter =  (firstfrostdayearlier-firstyeardate)+(lastyeardate2-lastfrostday)
                frfreeperiodshorter = frfreeperiodshorter.days
        else:
            if firstfrostdayearlier!=0:
                if latitude>0:                      
                    frfreeperiodshorter = lastyeardate-firstfrostdayearlier 
                    frfreeperiodshorter = yearlong-(frfreeperiodshorter.days+1)  
                else:                    
                    frfreeperiodshorter = firstfrostdayearlier-firstyeardate
                    frfreeperiodshorter = yearlong-(frfreeperiodshorter.days+1)
            elif lastfrostdaylater!=0:
                if latitude>0:
                    frfreeperiodshorter = lastfrostdaylater-firstyeardate
                    frfreeperiodshorter = yearlong-(frfreeperiodshorter.days+1)
                else:
                    frfreeperiodshorter = lastyeardate-lastfrostdaylater
                    frfreeperiodshorter = yearlong-(frfreeperiodshorter.days+1)
            else:
                frfreeperiodshorter=yearlong
        #frostfreeperiod unc
        frfreeperioduncup = abs(frostfreeperiod-frfreeperiodlonger)
        frfreeperioduncdown = abs(frostfreeperiod-frfreeperiodshorter) 
        #lastfrostday unc
        if lastfrostday!=0 or lastfrostdayearlier!=0:
            aux2 = lastfrostday-lastfrostdayearlier
            lfrostdayuncdown = aux2.days # earlier (number of days) 
        else:
            lfrostdayuncdown = 0
        if lastfrostday!=0 or lastfrostdaylater!=0:
            aux1= lastfrostdaylater-lastfrostday 
            lfrostdayuncup = aux1.days # later (number of days)
        else:
            lfrostdayuncup = 0
        #Firstfrostday unc    
        if firstfrostdaylater!=0 or firstfrostday!=0:
            aux1 = firstfrostdaylater-firstfrostday
            ffrostdayuncup = aux1.days # later (number of days)
        else:
            ffrostdayuncup = 0
        if firstfrostday!=0 or firstfrostdayearlier!=0:
            aux2 = firstfrostday-firstfrostdayearlier
            ffrostdayuncdown = aux2.days # earlier (number of days)
        else:
            ffrostdayuncdown = 0
        if lastfrostday!=0: # for printing
            sdaylong = str(lastfrostday)
            lfday = int(sdaylong[8:10])
            lfmonth = int(sdaylong[5:7])
        else:
            lfday = 0
            lfmonth = 0
        if firstfrostday!=0: # for printing        
            sdaylong = str(firstfrostday)
            ffday = int(sdaylong[8:10])
            ffmonth = int(sdaylong[5:7]) 
        else:
            ffday = 0
            ffmonth = 0         
    
    # day for fertilization - uncertainty:
    if calcfert == 1 :  # if we work with soil temperature
        if lastfertdaylater!=0 or lastfertday!=0:
            aux1 = lastfertdaylater-lastfertday
            lfertdayuncup = aux1.days # later (number of days)
        else:
            lfertdayuncup = 0
        if lastfertday!=0 or lastfertdayearlier!=0:
            aux2 = lastfertday-lastfertdayearlier
            lfertdayuncdown = aux2.days # earlier (number of days)
        else:
            lfertdayuncdown = 0
        if lastfertday!=0: # for printing  
            fdaylong = str(lastfertday)
            lfertday = int(fdaylong[8:10])
            lfertmonth = int(fdaylong[5:7]) 
        else: 
            lfertday = 0
            lfertmonth = 0
            
    #calculation of annual water balanc:    
    if calcwater == 1 :
        abalanc = apreci - aevapo  
        abalancunc = apreciunc + aevapounc
        if calcrunoff==1:
            abalanc -= arunoff
            abalancunc += arunoffunc
    #----------------------------------------------printing csv yearly--------------------------------- (
    if aprint == 1:
        ttext= str(latitude) + ", " + str(longitude) + ", " 
        ttext = ttext + str(tyear)         
        if calctemp == 1:
            ttext = ttext + ", " + str(numbfrostdays) + ", " + str(numbfrostdaysup) + ", " + str(numbfrostdaysdown) + ", " + str(lfday) + "." + str(lfmonth) + ", " + str(lfrostdayuncup) + ", " + str(lfrostdayuncdown) + ", " + str(ffday) + "." + str(ffmonth) + ", " + str(ffrostdayuncup) + ", " + str(ffrostdayuncdown)
            ttext = ttext + ", " + str(frostfreeperiod) + ", " + str(frfreeperioduncup) + ", " + str(frfreeperioduncdown)
        if calctempcrop == 1:
            ttext = ttext + ", " + str(numbofgrowday) + ", " + str(numbofgrowdayup) + ", " + str(numbofgrowdaydown) + ", " + str(numbofoptgrowday) + ", " + str(numbofoptgrowdayup) + ", " + str(numbofoptgrowdaydown) + ", " + str(round(agdu,2)) + ", " + str(round(agduuncup,2)) + ", " + str(round(agduuncdown,2)) + ", " + str(round(ahsu,2)) + ", " + str(round(ahsuuncup,2)) + ", " + str(round(ahsuuncdown,2))
        if calcsolar == 1:
            ttext = ttext + ", " +  str(round(asolar,2)) + ", " + str(round(asolarunc,2)) 
        if calcwater == 1:
            ttext = ttext + ", " +  str(round(abalanc,2)) + ", " + str(round(abalancunc,2)) 
            ttext = ttext + ", " +  str(round(apreci,2)) + ", " + str(round(apreciunc,2))  + ", " +  str(round(aevapo,2)) + ", " + str(round(aevapounc,2)) 
        if calcrunoff == 1: 
            ttext = ttext + ", " +  str(round(arunoff,2)) + ", " + str(round(arunoffunc,2)) 
        if calcfert == 1: 
            ttext = ttext + ", " +  str(lfertday) + "." + str(lfertmonth) + ", " + str(lfertdayuncup) + ", " + str(lfertdayuncdown)
        print_csv(exportfolder,fnameyearly, ttext, 0)
    #----------------------------------------------printing csv yearly--------------------------------- )

###  Function for more era points in one downloaded package and one year

In [11]:
def onepackageoneyear(stlat,stlon, enlat, enlon,year,exportfolder,eralatstep, eralonstep,fnameyearly,eradataset,edadataset,vartemp,varevapotranspiration,varrunoff,varsoiltemp,varsolar,varprecipitation,apifolder,frostdegree,unitcoeffsolar,unitcoeffwater,unitcoeffrunoff,dayinrow,starthourday,endhourday,hourstep,cropabsmax,cropabsmin,cropoptmax,cropoptmin,eda_t,estarthourday,eendhourday,edastep,dprint, wprint, aprint, eda_s, eda_f, eda_w, calctemp, calcfert, calcwater, calcrunoff, calctempcrop, fertdegree, dayinrowfert):
    #download input data through api:
    min_lat = str(stlat)
    max_lat = str(enlat)
    min_lon = str(stlon)
    max_lon = str(enlon)
    era5fname = "ERA5_" + min_lat[0:2]+"-"+ max_lat[0:2]+"-"+ min_lon[0:2]+"-"+ max_lon[0:2]+"-"+ str(year)+".nc"
    edafname = "EDA_" + min_lat[0:2]+"-"+ max_lat[0:2]+"-"+ min_lon[0:2]+"-"+ max_lon[0:2]+"-"+ str(year)+".nc"

    erapath = apifolder + "/" + era5fname
    edapath = apifolder + "/" + edafname  
    iseraFile = os.path.isfile(erapath) 
    isedaFile = os.path.isfile(edapath) 
    if iseraFile == False or isedaFile == False: #check if file doesn´t exist
        oneyearapi(year,max_lat,min_lat, max_lon, min_lon,era5fname,edafname,eradataset,edadataset,vartemp,varevapotranspiration, varrunoff, varsoiltemp, varsolar, varprecipitation,apifolder)
    
    #calculate points in one package:  
    tname = apifolder + "/" + edafname
    ie=Image(netCDF4.Dataset(tname,'r')) # creating Image object and reading data about factors into it        
    tname = apifolder + "/" + era5fname
    im=Image(netCDF4.Dataset(tname,'r')) # creating Image object and reading data about factors into it
    p1list = im.get_variables() 
    lonparam =  p1list[0]
    latparam =  p1list[1]
    timeparam =  p1list[2]
    tempparameter = p1list[3]
    evapoparameter = p1list[4]
    runoffparameter = p1list[5]
    soilparameter = p1list[6]
    solarparameter = p1list[7]
    preciparameter = p1list[8]

    p2list = ie.get_variables() 
    edalonparam =  p2list[0]
    edalatparam =  p2list[1]
    edatimeparam =  p2list[2]
    edatparam = p2list[3]
    edaeparam = p2list[4]
    edarparam = p2list[5]
    edafparam = p2list[6]
    edasparam = p2list[7]
    edapparam = p2list[8]
    lonlist = im.get_data().variables[lonparam][::] 
    latlist = im.get_data().variables[latparam][::] 
    timelist = im.get_data().variables[timeparam][::] 
    edalonlist = ie.get_data().variables[edalonparam][::] 
    edalatlist = ie.get_data().variables[edalatparam][::] 
    edatimelist = ie.get_data().variables[edatimeparam][::]    
    lonlistlen = len(lonlist)
    latlistlen = len(latlist)
    
    #create csv files:
    if dprint == 1:
        fnamedaily="daily-" + min_lat + max_lat + min_lon + max_lon + str(year)
        ttext= "latitude, longitude, "
        ttext= ttext + "day" + "." + "month" + "." + "year" 
        if calctemp == 1: 
            ttext= ttext + ", " +  "dayavg" + ", " + "davgunc" + ", " + "daymin" + ", " + "dminunc" + ", " + "daymax" + ", " + "dmaxunc"                                    
    print_csv(exportfolder,fnamedaily, ttext, 1)    
    
    if wprint == 1:
        fnameweekly="weekly-" + min_lat + max_lat + min_lon + max_lon + str(year)
        ttext= "latitude, longitude, "
        ttext= ttext + "year" +  ", " + "week"
        if calctempcrop == 1:
            ttext = ttext + ", " +  "wgdu" + ", " + "wgduuncup" + ", " + "wgduuncdown" + ", " + "whsu" + ", " + "whsuuncup" + ", " + "whsuuncdown"
        if calcsolar == 1:
            ttext = ttext + ", " +  "wsolar" + ", " + "wsolarunc" 
        if calcwater == 1:
            ttext = ttext + ", " +  "wpreci" + ", " + "wpreciunc" + ", " +  "wevapo" + ", " + "wevapounc" 
        if calcrunoff == 1: 
            ttext = ttext + ", " +  "wrunoff" + ", " + "wrunoffunc"           
        print_csv(exportfolder,fnameweekly, ttext, 1)    

       
    for lat in range(0, latlistlen-1, eralatstep): 
        for lon in range(0, lonlistlen-1, eralonstep):
            latitude = latlist[lat]
            longitude = lonlist[lon]
            findfacteda(latitude,longitude,year,fnameyearly,fnameweekly, fnamedaily,exportfolder,frostdegree,unitcoeffsolar,unitcoeffwater,unitcoeffrunoff,dayinrow,starthourday,endhourday,hourstep,cropabsmax,cropabsmin,cropoptmax,cropoptmin,im,ie,tempparameter,eda_t,estarthourday,eendhourday,edastep,edatparam,edalonparam, edalatparam, edatimeparam,latparam, lonparam, timeparam,edalatlist,edalonlist,dprint, wprint, aprint, eda_s, eda_f, eda_w, calctemp, calcfert, calcwater, calcrunoff, calctempcrop, soilparameter, edafparam, fertdegree, dayinrowfert, solarparameter, edasparam, preciparameter, edapparam, evapoparameter, edaeparam, runoffparameter, edarparam)
    
   
    

###  Function for more years and one downloaded package

In [12]:
def onepackagemoreyears(stlat,stlon, enlat, enlon, startyear,endyear,yearstep,exportfolder,eralatstep, eralonstep,fnameyearly,eradataset,edadataset,vartemp,varevapotranspiration,varrunoff,varsoiltemp,varsolar,varprecipitation,apifolder,frostdegree,unitcoeffsolar,unitcoeffwater,unitcoeffrunoff,dayinrow,starthourday,endhourday,hourstep,cropabsmax,cropabsmin,cropoptmax,cropoptmin,eda_t,estarthourday,eendhourday,edastep,dprint, wprint, aprint, eda_s, eda_f, eda_w, calctemp, calcfert, calcwater, calcrunoff, calctempcrop, fertdegree, dayinrowfert):
    for year in range (startyear, endyear+1,yearstep):
           onepackageoneyear(stlat,stlon, enlat, enlon,year,exportfolder,eralatstep, eralonstep,fnameyearly,eradataset,edadataset,vartemp,varevapotranspiration,varrunoff,varsoiltemp,varsolar,varprecipitation,apifolder,frostdegree,unitcoeffsolar,unitcoeffwater,unitcoeffrunoff,dayinrow,starthourday,endhourday,hourstep,cropabsmax,cropabsmin,cropoptmax,cropoptmin,eda_t,estarthourday,eendhourday,edastep,dprint, wprint, aprint, eda_s, eda_f, eda_w, calctemp, calcfert, calcwater, calcrunoff, calctempcrop, fertdegree, dayinrowfert)



###  Function for calculation of the whole area

In [13]:
def wholearea(startlat,startlon, endlat, endlon, startyear,endyear,yearstep,exportfolder,eralatstep, eralonstep,eradataset,edadataset,vartemp,varevapotranspiration,varrunoff,varsoiltemp,varsolar,varprecipitation,apifolder,packagesteplat,packagesteplon,frostdegree,unitcoeffsolar,unitcoeffwater,unitcoeffrunoff,dayinrow,starthourday,endhourday,hourstep,cropabsmax,cropabsmin,cropoptmax,cropoptmin,eda_t,estarthourday,eendhourday,edastep,dprint, wprint, aprint, eda_s, eda_f, eda_w, calctemp, calcfert, calcwater, calcrunoff, calctempcrop, fertdegree, dayinrowfert):
    if aprint == 1:
        fnameyearly="yearly"
        ttext= "latitude, longitude, "
        ttext= ttext + "year" 
        if calctemp == 1:
            ttext = ttext + ", " + "numbfrostdays" + ", " + "frostdaysuncup" + ", " + "numbfrostdaysuncdown" + ", " + "lfday" + "." + "lfmonth" + ", " + "lastfrostdayuncup" + ", " + "lastfrostdayuncdown" + ", " + "ffday" + "." + "ffmonth" + ", " + "ffrostdayuncup" + ", " + "firstfrostdayuncdown"
            ttext = ttext + ", " + "frostfreeperiod" + ", " + "frfreeperuncup" + ", " + "frfreeperuncdown"
        if calctempcrop == 1:
            ttext = ttext + ", " + "numbofgrowday" + ", " + "numbofgrowdayup" + ", " + "numbofgrowdaydown" + ", " + "numbofoptgrowday" + ", " + "numbofoptgrowdayup" + ", " + "numbofoptgrowdaydown" + ", " + "agdu" + ", " + "agduuncup" + ", " + "agduuncdown" + ", " + "ahsu" + ", " + "ahsuuncup" + ", " + "ahsuuncdown"
        if calcsolar == 1:
            ttext = ttext + ", " +  "asolar" + ", " + "asolarunc" 
        if calcwater == 1:
            ttext = ttext + ", " +  "abalanc" + ", " + "abalancunc" 
            ttext = ttext + ", " +  "apreci" + ", " + "apreciunc" + ", " +  "aevapo" + ", " + "aevapounc"  
        if calcrunoff == 1: 
            ttext = ttext + ", " +  "arunoff" + ", " + "arunoffunc" 
        if calcfert == 1: 
            ttext = ttext + ", " +  "lfertday" + "." + "lfertmonth" + ", " + "lastfertdayuncup" + ", " + "lastfertdayuncdown"
        print_csv(exportfolder,fnameyearly, ttext, 1)        
    for lat in np.arange(startlat,endlat,packagesteplat):
        for lon in np.arange(startlon,endlon, packagesteplon):
            stlat = lat
            enlat = lat+packagesteplat
            stlon = lon
            enlon = lon+packagesteplon
            onepackagemoreyears(stlat,stlon, enlat, enlon, startyear,endyear,yearstep, exportfolder,eralatstep, eralonstep,fnameyearly,eradataset,edadataset,vartemp,varevapotranspiration,varrunoff,varsoiltemp,varsolar,varprecipitation,apifolder,frostdegree,unitcoeffsolar,unitcoeffwater,unitcoeffrunoff,dayinrow,starthourday,endhourday,hourstep,cropabsmax,cropabsmin,cropoptmax,cropoptmin,eda_t,estarthourday,eendhourday,edastep,dprint, wprint, aprint, eda_s, eda_f, eda_w, calctemp, calcfert, calcwater, calcrunoff, calctempcrop, fertdegree, dayinrowfert)
   
            

### Select parameters and run script

In [ ]:
#select location (degrees):
startlat = 48.0
endlat = 52.0
startlon = 12.0
endlon = 18.0
#select api-download step (degrees):
packagesteplat = 2.0 # tested 1.0, 2.0, using higher value can be problem with limited size of request
packagesteplon = 2.0 # tested 1.0, 2.0, using higher value can be problem with limited size of request

#select years (integer):
startyear = 1982 # ERA5-Land start from 1982 (complete annual data)
endyear = 2019
yearstep = 1

#select calculated factors:
# !alg. limitation - missing optimalization of downloaded factors - always api downloads all factors (also not tested with missing input factors)
calctemp = 1 # calculate temperatures? 1=yes, 0=no #if calctemp=0, calctempcrop must be also 0
calcfert = 1 # calculate fall fertilization dates? 1=yes, 0=no
calcwater = 1 # calculate water balance? 1=yes, 0=no
calcrunoff = 1 # use runoff in water balance? 1=yes, 0=no # if 1, calcwater must be also 1
calcsolar = 1 # calculate solar radiation? 1=yes, 0=no 
eda_t= 1 # calculate eda for temperature? 1=yes, 0=no
eda_s = 1 # calculate eda for solar radiation? 1=yes, 0=no
eda_f = 1 # calculate eda for fertilizing/soil temperature? 1=yes, 0=no
eda_w = 1 # calculate eda for water balance? 1=yes, 0=no

calctempcrop = 1 # calculate gdu, hsu, number of (optimal) growing days? 1=yes, 0=no #if calctemp=0, calctempcrop must be also 0
#specification of temperatures for crop growing:
cropabsmax = 31 #f.e.: 31 = wheat - https://agritech.tnau.ac.in/agriculture/agri_agrometeorology_temp.html
cropabsmin = 3.5 #f.e.: 3.5 = wheat - https://agritech.tnau.ac.in/agriculture/agri_agrometeorology_temp.html
cropoptmax = 25 #f.e.: 25 = wheat - http://www.fao.org/3/y4011e06.htm
cropoptmin = 12 #f.e.: 12 = wheat - http://www.fao.org/3/y4011e06.htm

#specification of calculation:
starthourday = 0 # temperature data start hour
estarthourday = 0 # eda data start hour
endhourday = 23 # temperature data start hour
eendhourday = 21  # eda data end hour
eralatstep = 1 # step of era input data latitude (1=all latitudes)
eralonstep = 1 # step of era input data longitude (1=all longitudes)

#specification of agro-climatic factors:
frostdegree = 0
fertdegree = 10 #soil temperature for fall fertilization
dayinrow = 1 # elimination of (one-)daily fluctuations - last/first frost days in row
dayinrowfert = 1  # elimination of (one-)daily fluctuations -last days with soil temperature above X degree in row  

#specification of input data:
eradataset = 'reanalysis-era5-land'
edadataset = 'reanalysis-era5-single-levels'
vartemp = '2m_temperature'
varevapotranspiration = 'evaporation'
varrunoff = 'runoff'
varsoiltemp = 'soil_temperature_level_2'
varsolar = 'surface_net_solar_radiation'
varprecipitation = 'total_precipitation'
apifolder = "apidata"
hourstep = 1 # hourly temperature data # !alg. limitation - missing optimalization of requests of downloaded factors
edastep = 3 # eda data 3 hour step

#select output:            
dprint = 1 # daily exports? 1=yes, 0=no            
wprint = 1 # week exports? 1=yes, 0=no 
aprint = 1 # annual exports? 1=yes, 0=no 
exportfolder = "data" # 0 = same folder as script 

#Solar unit:
solarunits = 2  # 1 = J*m^(-2), 2 = MJ*m^(-2), 3 = Wh, 4 = kWh, usual value = 2
#Water balance unit:
waterunits = 3  # 1 = m, 2 = cm, 3 = mm (accuracy = 2 decimal values)

#--------------------------------------------------------------------------------------
unitcoeffsolar = 0.1 # 12/2020 ERA-Land - input data units in J*10 (in documentation J)
if solarunits == 2:
    unitcoeffsolar = 0.0000001
elif solarunits == 3: 
    unitcoeffsolar = 0.00278 
elif solarunits == 4: 
    unitcoeffsolar = 0.0000027778
    
    
unitcoeffwater = 0.1 # 12/2020 ERA-Land - input data units in decimeters (in documentation m)
if waterunits == 2:
    unitcoeffwater = 10
elif waterunits == 3: 
    unitcoeffwater = 100 
    
unitcoeffrunoff = unitcoeffwater

wholearea(startlat,startlon, endlat, endlon, startyear,endyear,yearstep,exportfolder,eralatstep, eralonstep,eradataset,edadataset,vartemp,varevapotranspiration,varrunoff,varsoiltemp,varsolar,varprecipitation,apifolder,packagesteplat,packagesteplon,frostdegree,unitcoeffsolar,unitcoeffwater,unitcoeffrunoff,dayinrow,starthourday,endhourday,hourstep,cropabsmax,cropabsmin,cropoptmax,cropoptmin,eda_t,estarthourday,eendhourday,edastep,dprint, wprint, aprint, eda_s, eda_f, eda_w, calctemp, calcfert, calcwater, calcrunoff, calctempcrop, fertdegree, dayinrowfert)
